In [67]:
# This is Eli's Python File
# Import statements
import numpy as np
import pandas as pd
from pathlib import Path
from MCForecastTools import MCSimulation
%matplotlib inline

In [11]:
# Read the affordability data
median_income_csv = Path("../Data/household_income_msa_2019.csv")
median_income_df = pd.read_csv(median_income_csv)
# Clean the affordability data
median_income_df = median_income_df.transpose() # Transpose the column/headers
new_header = median_income_df.iloc[0] #grab the first row for the header
median_income_df = median_income_df[1:] #take the data less the header row
median_income_df.columns = new_header #set the header row as the df header
median_income_df = median_income_df[['Median income (dollars)','Mean income (dollars)']] # Use only median and mean income data columns
median_income_df.reset_index(inplace=True)
median_income_df[['MSA','extra']] = median_income_df['index'].str.split(",", n = 1, expand=True) # Split MSA from text string
median_income_df[['State','extra2']] = median_income_df['extra'].str.split(n = 1, expand=True) # Split State from remaining text string
median_income_df[['Primary MSA','Secondary MSA']] = median_income_df['MSA'].str.split("-", n = 1, expand=True) # Split primary MSA from first text string
median_income_df[['Primary State','Secondary State']] = median_income_df['State'].str.split("-", n = 1, expand=True) # Split primary State from first text string
median_income_df.drop(columns=['index','extra','extra2','MSA','State'],inplace=True) # Drop unnecessary columns
median_income_df.drop_duplicates(subset=['Primary MSA','Primary State'], inplace=True) # Drop duplicates -- first instance is the median and mean income by MSA
median_income_df.head()

Label,Median income (dollars),Mean income (dollars),Primary MSA,Secondary MSA,Primary State,Secondary State
0,"61,026","77,948",Aberdeen,None,WA,None
8,"54,808","71,517",Abilene,None,TX,None
16,"53,865","69,443",Adrian,None,MI,None
24,"16,311","25,781",Aguadilla,Isabela,PR,None
32,"57,158","80,800",Akron,None,OH,None


In [88]:
# Testing Monte Carlo Simulation
# Read in test CSV
test_csv = Path("../Data/sim_test_data.csv")
test_df = pd.read_csv(test_csv)
test_df.set_index('Date', inplace=True)
test_df.columns = [test_df.columns,['Ticker'] * len(test_df.columns), ['close'] * len(test_df.columns)]
test_df.head()

,Test_MSA,Test_MSA2
,Ticker,Ticker
,close,close
Date,,
12/31/2019,100,100
1/31/2020,105,104
2/29/2020,102,103
3/31/2020,106,101
4/30/2020,110,99


In [120]:
MC_oneyear = MCSimulation(
    portfolio_data = test_df['Test_MSA'],
    weights = [1],
    num_simulation = 10,
    num_trading_days = 12*1
)

# Running a Monte Carlo simulation to forecast cumulative returns
sim_df = MC_oneyear.calc_cumulative_return() - 1
end_return = pd.DataFrame(index=['Average Cumulative Return'])
end_return['Test_MSA'] = sim_df.iloc[-1,:].mean()

Running Monte Carlo simulation number 0.


In [121]:
end_return

,Test_MSA
Average Cumulative Return,0.439731


In [124]:
# Configuring a Monte Carlo simulation to forecast five years cumulative returns
end_return = pd.DataFrame(index=['Average Cumulative Return'])
for msa in test_df.columns.get_level_values(0):

    MC_oneyear = MCSimulation(
        portfolio_data = test_df[msa],
        weights = [1],
        num_simulation = 10,
        num_trading_days = 12*1
    )

    # Running a Monte Carlo simulation to forecast cumulative returns
    sim_df = MC_oneyear.calc_cumulative_return() - 1
    end_return[msa] = sim_df.iloc[-1,:].mean()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 0.


In [125]:
end_return

,Test_MSA,Test_MSA2
Average Cumulative Return,0.29773,0.205301
